In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

xtest = pd.read_csv('X_test.csv')
xtrain = pd.read_csv('X_train.csv')
ytest = pd.read_csv("y_test.csv")
ytrain = pd.read_csv("y_train.csv")

df = pd.concat([xtest, xtrain], axis=0).sort_values(by='StudentID')
yresult = pd.concat([ytrain, ytest], axis=0).sort_values(by='StudentID')

df = pd.concat([df, yresult.drop(columns='StudentID')], axis=1).drop(columns='StudentID')


In [2]:
#one hot sklearn
from sklearn.preprocessing import OneHotEncoder

for i in df.columns:
    if(df[i].dtype in ['object', 'str']):
        df = pd.concat([pd.get_dummies(df[i], prefix=f'{i}').astype('int64'), df.drop(columns=i)], axis=1)

len(df.columns)
        

59

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

target = np.array(df['G3'])
features = StandardScaler().fit_transform(np.array(df.drop(columns='G3')))


xtrain, xtest, ytrain, ytest = train_test_split(features, target, random_state=23, test_size=0.2)

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

In [8]:
model = Sequential()

model.add(Dense(64, activation='relu', input_shape=(xtrain.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1))

model.compile(optimizer=Adam(lr=0.02), loss=tf.keras.losses.mae, metrics=[tf.keras.metrics.mae])


In [10]:
train = model.fit(
    xtrain, 
    ytrain,
    epochs=100, 
    batch_size=10,
    validation_data=(xtest, ytest)
)

Epoch 1/100
84/84 [==============================] - 0s 3ms/step - loss: 4.5605 - mean_absolute_error: 3.4589 - val_loss: 3.5337 - val_mean_absolute_error: 2.4265
Epoch 2/100
84/84 [==============================] - 0s 3ms/step - loss: 3.9735 - mean_absolute_error: 2.8659 - val_loss: 3.3366 - val_mean_absolute_error: 2.2296
Epoch 3/100
84/84 [==============================] - 0s 3ms/step - loss: 3.7087 - mean_absolute_error: 2.6050 - val_loss: 3.0361 - val_mean_absolute_error: 1.9368
Epoch 4/100
84/84 [==============================] - 0s 4ms/step - loss: 3.5721 - mean_absolute_error: 2.4783 - val_loss: 2.9013 - val_mean_absolute_error: 1.8142
Epoch 5/100
84/84 [==============================] - 0s 4ms/step - loss: 3.4978 - mean_absolute_error: 2.4173 - val_loss: 2.9276 - val_mean_absolute_error: 1.8546
Epoch 6/100
84/84 [==============================] - 0s 4ms/step - loss: 3.4712 - mean_absolute_error: 2.4058 - val_loss: 2.8338 - val_mean_absolute_error: 1.7763
Epoch 7/100
84/84 [===

In [11]:
model.predict(xtest)

# accuracy
ypred = model.predict(xtest)

def accuracy(ytest, ypred):
    right = 0
    total = 0
    for i, x in zip(ytest, ypred):
        if(i > x[0]-1 and i < x[0] + 1):
            right += 1
        total += 1
    return right/total

7/7 [==============================] - 0s 1ms/step


In [12]:
accuracy(ytest, ypred)

0.6507177033492823

In [13]:
for i, x in zip(ytest, ypred):
    print(i, round(x[0]))

13 11
12 11
13 12
11 12
11 8
13 13
14 13
14 12
16 16
15 15
15 15
9 9
16 15
8 8
15 16
18 17
9 9
9 9
11 10
11 12
11 12
16 16
16 16
13 14
10 12
14 15
0 10
12 12
12 11
15 15
13 13
13 11
11 11
4 5
13 12
11 11
14 15
14 13
10 10
11 11
7 7
9 9
12 11
0 4
11 11
16 16
13 13
13 11
14 13
11 12
12 11
10 11
14 11
15 13
13 13
14 14
15 16
11 11
16 17
8 6
11 11
13 14
10 11
0 9
11 10
15 15
13 12
15 15
13 12
11 12
10 11
13 13
14 15
11 11
9 9
16 17
12 11
13 12
11 12
11 10
17 16
10 10
14 14
8 10
15 13
9 8
14 12
11 11
10 8
11 12
11 12
13 15
12 12
10 10
10 8
13 13
8 7
13 14
10 9
11 10
16 16
9 9
12 12
10 9
17 16
10 10
18 17
12 12
13 13
13 13
11 11
10 12
13 13
9 11
10 12
13 13
16 16
16 15
12 13
12 12
11 9
13 13
16 17
9 8
10 10
11 11
12 13
10 11
14 14
18 17
17 17
18 16
13 13
10 10
12 13
12 10
11 11
14 13
16 16
10 11
10 11
12 11
10 10
0 6
13 12
0 9
12 10
11 10
10 9
15 14
11 9
11 10
17 17
12 12
8 5
10 11
14 14
18 17
11 11
10 10
6 8
14 14
0 7
10 10
10 9
10 12
12 10
0 -2
15 15
6 6
11 12
12 11
15 15
12 12
0 2
15 15
8

In [14]:
for i, x in zip(ytest, ypred):
    print(i, x[0])

13 11.295912
12 10.878537
13 12.413212
11 12.461786
11 7.777874
13 12.615528
14 13.127453
14 12.003542
16 15.9162
15 15.285161
15 14.913845
9 9.0946045
16 15.124886
8 7.8360744
15 15.790635
18 16.56025
9 8.631863
9 9.370045
11 10.1228485
11 12.285107
11 11.574861
16 15.906021
16 16.135456
13 14.038907
10 11.763596
14 14.696121
0 9.731533
12 12.048187
12 11.335095
15 14.613846
13 13.030525
13 11.490156
11 10.759514
4 5.2035704
13 11.841465
11 10.896719
14 14.75732
14 12.636949
10 10.010921
11 10.994352
7 7.3425303
9 8.588796
12 11.340759
0 3.9149523
11 10.890598
16 15.84423
13 12.59436
13 11.11836
14 13.309439
11 11.82359
12 11.022249
10 10.673183
14 11.293715
15 12.952293
13 12.923233
14 14.327515
15 15.937141
11 10.815065
16 16.59523
8 6.4414735
11 10.626641
13 13.569984
10 10.500208
0 9.46936
11 10.284539
15 15.415836
13 11.840929
15 14.912266
13 12.395558
11 11.983296
10 11.302204
13 12.684507
14 14.5359745
11 10.975203
9 8.606224
16 16.564255
12 10.7860155
13 11.998035
11 11.768808